In [1]:
!pip install --quiet langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [2]:
import os

os.environ["OPENAI_API_TYPE"] = "azure_ad"
os.environ["OPENAI_API_VERSION"] = "2024-05-01-preview"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "This form the Azure portal - model deployment"
# os.environ["AZURE_OPENAI_API_KEY"] = 'This form the Azure portal - model deployment'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://mobility-sage-openai-services.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = '9bbf92a17be24c66881eaa5a1c6aa77d'

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain.chains.router import MultiPromptChain
from langchain.chains import LLMChain
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
llm = AzureChatOpenAI(
            temperature=1,
            azure_deployment="gpt-4o",
        )

# Router Chain

A good way to imagine this is if you have multiple subchains, each of which is specialized for a particular type of input, you could have a router chain, which first decides which subchain to pass it to and then passes it to that chain.

In simple terms if we want out prompt to wear multiple hats or assume differnt roles based on the user input, we can use the router chain.

In [89]:
# Create multiple prompts

python_code_prompt="""
  You can do coding only and only in python. You are not allowed to code in any other language. \
  If someone asks you to code in python then do it otherwise answer politely that I am only allowed to code \
  in python nothing else. Now Code in python for below problem statement delimited by triple ticks\n \
  ```{input}```
"""


cpp_code_prompt= """
  You can do coding only and only in C++. You are not allowed to code in any other language. \
  If someone asks you to code in C++ then do it otherwise answer politely that I am only allowed to code \
  in C++ nothing else. Now Code in C++ for below problem statement delimited by triple ticks\n \
  ```{input}```
"""


sql_code_prompt= """
  You can do coding only and only in SQL. You are not allowed to code in any other language. \
  If someone asks you to code in SQL then do it otherwise answer politely that I am only allowed to code \
  in SQL nothing else. Now Code in SQL for below problem statement delimited by triple ticks\n \
  ```{input}```
"""

In [90]:
# Build a prompt descriptor

prompt_infos = [
    {
          "name": "Python Expert",
          "description": "This chain is expert to code in python only",
          "prompt_template": python_code_prompt
      },
      {
          "name": "C++ Expert",
          "description": "This chain is expert to code in C++ language only",
          "prompt_template": cpp_code_prompt
      },

      {
          "name": "SQL Expert",
          "description": "This chain is expert to code in SQL language only",
          "prompt_template": sql_code_prompt
      }
]

## Build the Destination chains.

## These are the chains that will be called by the router chain.

In [91]:
destination_chain = {}

for p_info in prompt_infos:
  name = p_info['name']
  prompt_template = p_info['prompt_template']

  prompt = PromptTemplate(
      template=prompt_template,
      input_variables=['input']
  )

  chain = LLMChain(
      llm = llm,
      prompt = prompt
  )

  destination_chain[name] = chain



In [92]:
destination_chain.keys()

dict_keys(['Python Expert', 'C++ Expert', 'SQL Expert'])

In [93]:
default_prompt = PromptTemplate(
    template="Understand the input and return the code against the below user input. You are only allowed to code nothing else.\n {input}",
    input_variables=["input"]
    )

In [94]:
default_chain = LLMChain(
    llm = llm,
    prompt = default_prompt,
    output_key = "text"
)

In [95]:
default_prompt = PromptTemplate.from_template("{blackhole}")
default_prompt

PromptTemplate(input_variables=['blackhole'], template='{blackhole}')

In [96]:
destinations = [f"{p['name']} : {p['description']}" for p in prompt_infos]
destinations

['Python Expert : This chain is expert to code in python only',
 'C++ Expert : This chain is expert to code in C++ language only',
 'SQL Expert : This chain is expert to code in SQL language only']

In [97]:
destinations_str = '\n'.join(destinations)
destinations_str

'Python Expert : This chain is expert to code in python only\nC++ Expert : This chain is expert to code in C++ language only\nSQL Expert : This chain is expert to code in SQL language only'

# Router Chain.

In [98]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [99]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destinations_str
)

print(router_template) # built in  MULTI_PROMPT_ROUTER_TEMPLATE from langchain

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Python Expert : This chain is expert to code in python only
C++ Expert : This chain is expert to code

In [100]:
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

In [108]:
router_chain = LLMRouterChain.from_llm(
    llm=llm,
    prompt=router_prompt
)

In [109]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chain,
    default_chain=default_chain,
    verbose=True
)

In [110]:
language='python'

prefix_prompt=f'''Write code in {language} language. '''
input_prompt='''Take two sorted arrays [3,6,7,9] and [4,5,6,8,10]. Make one sorted array from them'''

final_prompt= prefix_prompt + input_prompt


In [111]:
chain.run({
    "input": final_prompt,
    "destinations": destinations_str
    })



> Entering new MultiPromptChain chain...
Python Expert: {'input': 'Write code in python language. Take two sorted arrays [3,6,7,9] and [4,5,6,8,10]. Make one sorted array from them'}
> Finished chain.


'Certainly! Here is the Python code to merge two sorted arrays into one sorted array:\n\n```python\ndef merge_sorted_arrays(arr1, arr2):\n    merged_array = []\n    i = 0\n    j = 0\n\n    while i < len(arr1) and j < len(arr2):\n        if arr1[i] < arr2[j]:\n            merged_array.append(arr1[i])\n            i += 1\n        else:\n            merged_array.append(arr2[j])\n            j += 1\n\n    # Append any remaining elements from arr1\n    while i < len(arr1):\n        merged_array.append(arr1[i])\n        i += 1\n\n    # Append any remaining elements from arr2\n    while j < len(arr2):\n        merged_array.append(arr2[j])\n        j += 1\n\n    return merged_array\n\n# Sample arrays\narr1 = [3, 6, 7, 9]\narr2 = [4, 5, 6, 8, 10]\n\n# Get the merged array\nmerged_array = merge_sorted_arrays(arr1, arr2)\nprint(merged_array)\n```\n\nThis code defines a function `merge_sorted_arrays` that takes two sorted arrays as input and merges them into a single sorted array. The function the

In [112]:
print(chain.run("What is the first prime number greater than 40 such that one plus the prime number is divisible by 3"))



> Entering new MultiPromptChain chain...
None: {'input': 'What is the first prime number greater than 40 such that one plus the prime number is divisible by 3'}
> Finished chain.
```python
def is_prime(num):
    if num <= 1:
        return False
    for i in range(2, int(num**0.5) + 1):
        if num % i == 0:
            return False
    return True

number = 41
while True:
    if is_prime(number) and (number + 1) % 3 == 0:
        print(number)
        break
    number += 1
```


In [113]:
print(chain.run("Write a python code to add 2 numbers"))



> Entering new MultiPromptChain chain...
Python Expert: {'input': 'Write a Python code to add 2 numbers'}
> Finished chain.
Sure, here's a simple Python code to add two numbers:

```python
def add_two_numbers(num1, num2):
    return num1 + num2

# Example usage:
number1 = 5
number2 = 3
result = add_two_numbers(number1, number2)
print("The sum of {} and {} is {}".format(number1, number2, result))
```

This function `add_two_numbers` takes two arguments and returns their sum. In the example usage, it adds 5 and 3 to get 8.


In [117]:
print(chain.run({
    "input": 'Search for values in a database',
    "destinations": destinations_str
    }))



> Entering new MultiPromptChain chain...
SQL Expert: {'input': 'Search for values in a SQL database'}
> Finished chain.
```SQL
-- This query will search for values in a specific column of a table in a SQL database.

-- Replace `table_name` with the name of your table
-- Replace `column_name` with the name of the column you want to search in
-- Replace `search_value` with the value you are searching for

SELECT *
FROM table_name
WHERE column_name = 'search_value';
```

This query selects all columns from the specified table where the value in the specified column matches the search value.
